In [1]:
import sys
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy import interpolate 
sys.path.append(r'C:\Chuji\Code_and_Data\MyCode')

import Circuit_Simulator 
import PulseGenerator as PG
from toolfunc import *
from toolfunc.adia_analysis import *
from toolfunc.pulse_filter import *
import scipy.optimize as sci_op
from toolfunc import DE

In [52]:
raw_config = Circuit_Simulator.RawConfig(qubit_num=3,dimension=3,circuit_type=1,initial_state='ground',sampling_rate=1e9)
raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)

flux_pulse = np.linspace(0/7.5,4/7.5,400)
freq_array = 8.5e9-flux_pulse * (8.5e9 - 1e9)

raw_config.setValue('Q1 f01_max',6.0e9)
raw_config.setValue('Q2 f01_max',8.5e9)
raw_config.setValue('Q3 f01_max',5.4e9)
raw_config.setValue('Q1 f01_min',1e9)
raw_config.setValue('Q2 f01_min',1e9)
raw_config.setValue('Q3 f01_min',1e9)
raw_config.setValue('Q1 Ec',0.25e9)
raw_config.setValue('Q2 Ec',0.3e9)
raw_config.setValue('Q3 Ec',0.25e9)
raw_config.setValue('r12',0.03)
raw_config.setValue('r23',0.03)
raw_config.setValue('r13',0.0036)

raw_config.setValue('Q2 Voltage period',-1)
raw_config.setValue('Q2 Voltage operating point',0.00)
raw_config.setValue('Q2 Flux',flux_pulse)

simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
Simulator = Circuit_Simulator.Simulator(simu_config)
Simulator.show_pulse()

{'Q1_freq': '6000.0MHz', 'Q1_anhar': '-125.0MHz', 'Q2_freq': '8500.0MHz', 'Q2_anhar': '-150.0MHz', 'Q3_freq': '5400.0MHz', 'Q3_anhar': '-125.0MHz', 'g12': '214.2429MHz', 'g13': '20.4916MHz', 'g23': '203.2486MHz'}


In [53]:
Simulator.performsimulation(solver_type=2,resample_factor=1,eigen_cloest_to_bare=False ,sort_by_maximum_overlap=True,gap=12e6)

fig = plt.figure(figsize=[6.4,6])
ax = fig.add_subplot(111)

eigen_trace = Simulator.EigenResult.get_Ener_gap_trace('101-100-001+000')
ax.plot(freq_array[0:400],-eigen_trace[0:400]/1e6)

ax.set_yscale('log')

In [6]:
%matplotlib qt

In [ ]:
#     raw_config.setValue('Q1 f01_max',6.0e9)
#     raw_config.setValue('Q2 f01_max',7.87e9)
#     raw_config.setValue('Q3 f01_max',5.4e9)
#     raw_config.setValue('Q1 f01_min',1e9)
#     raw_config.setValue('Q2 f01_min',1e9)
#     raw_config.setValue('Q3 f01_min',1e9)
#     raw_config.setValue('Q1 Ec',0.25e9)
#     raw_config.setValue('Q2 Ec',0.3e9)
#     raw_config.setValue('Q3 Ec',0.25e9)
#     raw_config.setValue('r12',0.018)
#     raw_config.setValue('r23',0.018)
#     raw_config.setValue('r13',0.0015)

    # raw_config.setValue('Q1 f01_max',6.0e9)
    # raw_config.setValue('Q2 f01_max',8.3e9)
    # raw_config.setValue('Q3 f01_max',5.4e9)
    # raw_config.setValue('Q1 f01_min',1e9)
    # raw_config.setValue('Q2 f01_min',1e9)
    # raw_config.setValue('Q3 f01_min',1e9)
    # raw_config.setValue('Q1 Ec',0.25e9)
    # raw_config.setValue('Q2 Ec',0.30e9)
    # raw_config.setValue('Q3 Ec',0.25e9)
# raw_config.setValue('r12',0.03)
# raw_config.setValue('r23',0.03)
# raw_config.setValue('r13',0.0036)

In [4]:
%matplotlib inline

In [45]:
def cost_func_distor_20(pulse_params,*args):
    gate_time,SRATE,f_term,factor_r,T_reflec=args
    lamb1 = pulse_params
    total_len = gate_time + 8e-9+4*T_reflec
    
    Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
    Seq.clear_pulse(tips_on=False)
    Seq.add_pulse('Adiabatic',t0=gate_time/2+10e-9/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=f_term,Lcoeff=np.array(lamb1),Q1_freq=6.0e9,
                 CPLR_idle_freq=8.16e9,Q2_freq=5.4e9,constant_coupling=False,r1c=0.02,r2c=0.02,r12=0.002,anhar_CPLR=-300e6,
                 anhar_Q1=-250e6,anhar_Q2=-250e6,negative_amplitude=False,dfdV=8.16e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
    Seq.add_filter('Gauss Low Pass',300e6)
    Seq.add_filter('Reflection',*(factor_r,T_reflec))
    flux_pulse=Seq.get_sequence()

    raw_config = Circuit_Simulator.RawConfig(qubit_num=3,dimension=3,circuit_type=1,initial_state='-Z+Z+Z',sampling_rate=SRATE)
    raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)
    
    raw_config.setValue('Q1 f01_max',6.0e9)
    raw_config.setValue('Q2 f01_max',8.16e9)
    raw_config.setValue('Q3 f01_max',5.4e9)
    raw_config.setValue('Q1 f01_min',1e9)
    raw_config.setValue('Q2 f01_min',1e9)
    raw_config.setValue('Q3 f01_min',1e9)
    raw_config.setValue('Q1 Ec',0.25e9)
    raw_config.setValue('Q2 Ec',0.3e9)
    raw_config.setValue('Q3 Ec',0.25e9)
    raw_config.setValue('r12',0.02)
    raw_config.setValue('r23',0.02)
    raw_config.setValue('r13',0.002)
    raw_config.setValue('Q2 Voltage period',-1)
    raw_config.setValue('Q2 Voltage operating point',0)
    raw_config.setValue('Q2 Flux',flux_pulse)

    simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
    Simulator = Circuit_Simulator.Simulator(simu_config)

    Simulator.performsimulation(solver_type=1)
    Simulator.UnitaryResult.get_U(-1)
    Simulator.UnitaryResult.get_subspace_operator(['000','001','100','101'])
    Simulator.UnitaryResult.set_Target_gate('CZ')
    Simulator.UnitaryResult.remove_single_qubit_gate()
    Simulator.UnitaryResult.get_Gate_Fidelity()
    fidelity = Simulator.UnitaryResult.Gate_Fidelity
    return 1 - fidelity



def cost_func_distor_12(pulse_params,*args):
    gate_time,SRATE,f_term,factor_r,T_reflec=args
    lamb1 = pulse_params
    total_len = gate_time + 8e-9+4*T_reflec
    
    Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
    Seq.clear_pulse(tips_on=False)
    Seq.add_pulse('Adiabatic',t0=gate_time/2+10e-9/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=f_term,Lcoeff=np.array(lamb1),Q1_freq=6.0e9,
                 CPLR_idle_freq=8.2e9,Q2_freq=5.4e9,constant_coupling=False,r1c=0.012,r2c=0.012,r12=0.0006,anhar_CPLR=-300e6,
                 anhar_Q1=-250e6,anhar_Q2=-250e6,negative_amplitude=False,dfdV=8.2e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
    Seq.add_filter('Gauss Low Pass',300e6)
    Seq.add_filter('Reflection',*(factor_r,T_reflec))
    flux_pulse=Seq.get_sequence()

    raw_config = Circuit_Simulator.RawConfig(qubit_num=3,dimension=3,circuit_type=1,initial_state='-Z+Z+Z',sampling_rate=SRATE)
    raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)
    
    raw_config.setValue('Q1 f01_max',6.0e9)
    raw_config.setValue('Q2 f01_max',8.2e9)
    raw_config.setValue('Q3 f01_max',5.4e9)
    raw_config.setValue('Q1 f01_min',1e9)
    raw_config.setValue('Q2 f01_min',1e9)
    raw_config.setValue('Q3 f01_min',1e9)
    raw_config.setValue('Q1 Ec',0.25e9)
    raw_config.setValue('Q2 Ec',0.3e9)
    raw_config.setValue('Q3 Ec',0.25e9)
    raw_config.setValue('r12',0.012)
    raw_config.setValue('r23',0.012)
    raw_config.setValue('r13',0.0006)
    raw_config.setValue('Q2 Voltage period',-1)
    raw_config.setValue('Q2 Voltage operating point',0)
    raw_config.setValue('Q2 Flux',flux_pulse)

    simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
    Simulator = Circuit_Simulator.Simulator(simu_config)

    Simulator.performsimulation(solver_type=1)
    Simulator.UnitaryResult.get_U(-1)
    Simulator.UnitaryResult.get_subspace_operator(['000','001','100','101'])
    Simulator.UnitaryResult.set_Target_gate('CZ')
    Simulator.UnitaryResult.remove_single_qubit_gate()
    Simulator.UnitaryResult.get_Gate_Fidelity()
    fidelity = Simulator.UnitaryResult.Gate_Fidelity
    return 1 - fidelity



def cost_func_distor_30(pulse_params,*args):
    gate_time,SRATE,f_term,factor_r,T_reflec=args
    lamb1 = pulse_params
    total_len = gate_time + 8e-9+4*T_reflec
    
    Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
    Seq.clear_pulse(tips_on=False)
    Seq.add_pulse('Adiabatic',t0=gate_time/2+10e-9/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=f_term,Lcoeff=np.array(lamb1),Q1_freq=6.0e9,
                 CPLR_idle_freq=8.3e9,Q2_freq=5.4e9,constant_coupling=False,r1c=0.03,r2c=0.03,r12=0.0036,anhar_CPLR=-300e6,
                 anhar_Q1=-250e6,anhar_Q2=-250e6,negative_amplitude=False,dfdV=8.3e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
    Seq.add_filter('Gauss Low Pass',300e6)
    Seq.add_filter('Reflection',*(factor_r,T_reflec))
    flux_pulse=Seq.get_sequence()

    raw_config = Circuit_Simulator.RawConfig(qubit_num=3,dimension=3,circuit_type=1,initial_state='-Z+Z+Z',sampling_rate=SRATE)
    raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)
    
    raw_config.setValue('Q1 f01_max',6.0e9)
    raw_config.setValue('Q2 f01_max',8.3e9)
    raw_config.setValue('Q3 f01_max',5.4e9)
    raw_config.setValue('Q1 f01_min',1e9)
    raw_config.setValue('Q2 f01_min',1e9)
    raw_config.setValue('Q3 f01_min',1e9)
    raw_config.setValue('Q1 Ec',0.25e9)
    raw_config.setValue('Q2 Ec',0.3e9)
    raw_config.setValue('Q3 Ec',0.25e9)
    raw_config.setValue('r12',0.03)
    raw_config.setValue('r23',0.03)
    raw_config.setValue('r13',0.0036)
    raw_config.setValue('Q2 Voltage period',-1)
    raw_config.setValue('Q2 Voltage operating point',0)
    raw_config.setValue('Q2 Flux',flux_pulse)

    simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
    Simulator = Circuit_Simulator.Simulator(simu_config)

    Simulator.performsimulation(solver_type=1)
    Simulator.UnitaryResult.get_U(-1)
    Simulator.UnitaryResult.get_subspace_operator(['000','001','100','101'])
    Simulator.UnitaryResult.set_Target_gate('CZ')
    Simulator.UnitaryResult.remove_single_qubit_gate()
    Simulator.UnitaryResult.get_Gate_Fidelity()
    fidelity = Simulator.UnitaryResult.Gate_Fidelity
    return 1 - fidelity

def cost_func_distor_18(pulse_params,*args):
    gate_time,SRATE,f_term,factor_r,T_reflec=args
    lamb1 = pulse_params
    total_len = gate_time + 8e-9+4*T_reflec
    
    Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
    Seq.clear_pulse(tips_on=False)
    Seq.add_pulse('Adiabatic',t0=gate_time/2+10e-9/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=f_term,Lcoeff=np.array(lamb1),Q1_freq=6.0e9,
                 CPLR_idle_freq=7.87e9,Q2_freq=5.4e9,constant_coupling=False,r1c=0.018,r2c=0.018,r12=0.0015,anhar_CPLR=-300e6,
                 anhar_Q1=-250e6,anhar_Q2=-250e6,negative_amplitude=False,dfdV=7.87e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
    Seq.add_filter('Gauss Low Pass',300e6)
    Seq.add_filter('Reflection',*(factor_r,T_reflec))
    flux_pulse=Seq.get_sequence()

    raw_config = Circuit_Simulator.RawConfig(qubit_num=3,dimension=3,circuit_type=1,initial_state='-Z+Z+Z',sampling_rate=SRATE)
    raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)
    
    raw_config.setValue('Q1 f01_max',6.0e9)
    raw_config.setValue('Q2 f01_max',7.87e9)
    raw_config.setValue('Q3 f01_max',5.4e9)
    raw_config.setValue('Q1 f01_min',1e9)
    raw_config.setValue('Q2 f01_min',1e9)
    raw_config.setValue('Q3 f01_min',1e9)
    raw_config.setValue('Q1 Ec',0.25e9)
    raw_config.setValue('Q2 Ec',0.3e9)
    raw_config.setValue('Q3 Ec',0.25e9)
    raw_config.setValue('r12',0.018)
    raw_config.setValue('r23',0.018)
    raw_config.setValue('r13',0.0015)
    raw_config.setValue('Q2 Voltage period',-1)
    raw_config.setValue('Q2 Voltage operating point',0)
    raw_config.setValue('Q2 Flux',flux_pulse)

    simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
    Simulator = Circuit_Simulator.Simulator(simu_config)

    Simulator.performsimulation(solver_type=1)
    Simulator.UnitaryResult.get_U(-1)
    Simulator.UnitaryResult.get_subspace_operator(['000','001','100','101'])
    Simulator.UnitaryResult.set_Target_gate('CZ')
    Simulator.UnitaryResult.remove_single_qubit_gate()
    Simulator.UnitaryResult.get_Gate_Fidelity()
    fidelity = Simulator.UnitaryResult.Gate_Fidelity
    return 1 - fidelity

In [10]:
 np.arange(-0.095,0.1,0.005)

array([-9.50000000e-02, -9.00000000e-02, -8.50000000e-02, -8.00000000e-02,
       -7.50000000e-02, -7.00000000e-02, -6.50000000e-02, -6.00000000e-02,
       -5.50000000e-02, -5.00000000e-02, -4.50000000e-02, -4.00000000e-02,
       -3.50000000e-02, -3.00000000e-02, -2.50000000e-02, -2.00000000e-02,
       -1.50000000e-02, -1.00000000e-02, -5.00000000e-03,  8.32667268e-17,
        5.00000000e-03,  1.00000000e-02,  1.50000000e-02,  2.00000000e-02,
        2.50000000e-02,  3.00000000e-02,  3.50000000e-02,  4.00000000e-02,
        4.50000000e-02,  5.00000000e-02,  5.50000000e-02,  6.00000000e-02,
        6.50000000e-02,  7.00000000e-02,  7.50000000e-02,  8.00000000e-02,
        8.50000000e-02,  9.00000000e-02,  9.50000000e-02])

In [13]:
SRATE=6e9
gate_time=30e-9
f_terms=1
T_r=5e-9

factor_r_arr =  np.arange(-0.095,0.1,0.005)
gate_fidelity_one = np.zeros([len(factor_r_arr)])
gate_params_one = np.zeros([len(factor_r_arr)])
raw_initial_seeds=np.array([0.8])


jj = 0
for factor_r in factor_r_arr:
    time_start = time.time()
    DATA = sci_op.minimize(cost_func_distor_30,raw_initial_seeds,args=(gate_time,SRATE,f_terms,factor_r,T_r), method='Nelder-Mead', options={'disp': True,'ftol':1e-4,'xtol':1e-4,'maxiter':30})

    gate_fidelity_one[jj] = DATA.fun
    gate_params_one[jj] = DATA.x

    print('fidelity',DATA.fun)
    print(time.time()-time_start)
    np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\param_r_ic30_s.txt',gate_params_one )
    np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\_r_ic30_s.txt',gate_fidelity_one )
    jj+=1

Optimization terminated successfully.
         Current function value: 0.002476
         Iterations: 10
         Function evaluations: 20
fidelity 0.00247590676946563
39.8377161026001
Optimization terminated successfully.
         Current function value: 0.002069
         Iterations: 10
         Function evaluations: 20
fidelity 0.002069022919958363
39.76981973648071
Optimization terminated successfully.
         Current function value: 0.001722
         Iterations: 10
         Function evaluations: 20
fidelity 0.0017217496005375033
39.2352340221405
Optimization terminated successfully.
         Current function value: 0.001428
         Iterations: 10
         Function evaluations: 20
fidelity 0.0014280944019821051
39.27809953689575
Optimization terminated successfully.
         Current function value: 0.001181
         Iterations: 10
         Function evaluations: 20
fidelity 0.0011813824684961949
39.16103649139404
Optimization terminated successfully.
         Current function value:

In [24]:
SRATE=6e9
gate_time=30e-9
f_terms=1
T_r=5e-9

factor_r_arr = np.linspace(-0.1,0.1,41)
gate_fidelity_one = np.zeros([len(factor_r_arr)])
gate_params_one = np.zeros([len(factor_r_arr)])
raw_initial_seeds=np.array([0.8])


jj = 0
for factor_r in factor_r_arr:
    time_start = time.time()
    DATA = sci_op.minimize(cost_func_distor_20,raw_initial_seeds,args=(gate_time,SRATE,f_terms,factor_r,T_r), method='Nelder-Mead', options={'disp': True,'ftol':1e-4,'xtol':1e-4,'maxiter':30})

    gate_fidelity_one[jj] = DATA.fun
    gate_params_one[jj] = DATA.x
    raw_initial_seeds = DATA.x
    
    print('fidelity',DATA.fun)
    print(time.time()-time_start)
    np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\param_r_ic20.txt',gate_params_one )
    np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\error_r_ic20.txt',gate_fidelity_one )
    jj+=1

Optimization terminated successfully.
         Current function value: 0.004030
         Iterations: 12
         Function evaluations: 24
fidelity 0.0040302594293150085
42.69458842277527
Optimization terminated successfully.
         Current function value: 0.003809
         Iterations: 10
         Function evaluations: 20
fidelity 0.0038087216668524215
34.770692348480225
Optimization terminated successfully.
         Current function value: 0.003595
         Iterations: 10
         Function evaluations: 20
fidelity 0.0035953153542346294
33.93322229385376
Optimization terminated successfully.
         Current function value: 0.003385
         Iterations: 10
         Function evaluations: 20
fidelity 0.0033850473865948416
33.48358368873596
Optimization terminated successfully.
         Current function value: 0.003176
         Iterations: 10
         Function evaluations: 20
fidelity 0.00317572073957173
34.506200075149536
Optimization terminated successfully.
         Current function v

In [47]:
SRATE=6e9
gate_time=30e-9
f_terms=1
T_r=5e-9

factor_r_arr = np.linspace(-0.1,0.1,41)
gate_fidelity_one = np.zeros([len(factor_r_arr)])
gate_params_one = np.zeros([len(factor_r_arr)])
raw_initial_seeds=np.array([0.8])


jj = 0
for factor_r in factor_r_arr:
    time_start = time.time()
    DATA = sci_op.minimize(cost_func_distor_12,raw_initial_seeds,args=(gate_time,SRATE,f_terms,factor_r,T_r), method='Nelder-Mead', options={'disp': True,'ftol':1e-4,'xtol':1e-4,'maxiter':30})

    gate_fidelity_one[jj] = DATA.fun
    gate_params_one[jj] = DATA.x
    raw_initial_seeds = DATA.x
    
    print('fidelity',DATA.fun)
    print(time.time()-time_start)
    np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\param_r_ic12_new.txt',gate_params_one )
    np.savetxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\error_r_ic12_new.txt',gate_fidelity_one )
    jj+=1

Optimization terminated successfully.
         Current function value: 0.112086
         Iterations: 15
         Function evaluations: 30
fidelity 0.1120859822084378
50.70479774475098
Optimization terminated successfully.
         Current function value: 0.110699
         Iterations: 11
         Function evaluations: 22
fidelity 0.11069900364423013
37.29634499549866
Optimization terminated successfully.
         Current function value: 0.107985
         Iterations: 11
         Function evaluations: 22
fidelity 0.10798489711199322
36.640655517578125
Optimization terminated successfully.
         Current function value: 0.102673
         Iterations: 11
         Function evaluations: 22
fidelity 0.10267338860924191
36.66017770767212
Optimization terminated successfully.
         Current function value: 0.094188
         Iterations: 11
         Function evaluations: 22
fidelity 0.0941881649805788
36.58289074897766
Optimization terminated successfully.
         Current function value: 0.082

In [56]:
params = array([0.8846875 , 0.90784143, 0.92051929, 0.92869969, 0.93749694,
       0.94848324, 0.95839414, 0.96597519, 0.97172954, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

error = array([0.01201456, 0.05103845, 0.10193811, 0.131405  , 0.12629882,
       0.09646129, 0.05970911, 0.02951301, 0.01071783, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

array([0.8846875 , 0.90784143, 0.92051929, 0.92869969, 0.93749694,
       0.94848324, 0.95839414, 0.96597519, 0.97172954, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [58]:
np.linspace(-0.1,0.1,21)[9:]

array([-0.01,  0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,
        0.08,  0.09,  0.1 ])

In [60]:
# Error_two_5ns = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\error_two_distor_5ns.txt')
# Param_two_5ns = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\param_two_distor_5ns.txt')
Error_one_5ns = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\error_one_distor_5ns.txt')
Error_30_5ns = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\error_r_ic30.txt')
Error_09_5ns = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\error_r_ic09.txt')
Error_60ns_09_5ns = np.loadtxt(r'C:\Chuji\Latex_Papers\Mypapers\ZZ_coupling_20210205\fig_zz\Robustness\robustness_versus_coupling_strength\error_60ns_r_ic09.txt')

In [64]:
import matplotlib.ticker as mtick
fig = plt.figure(figsize=[7.6,4.8])
ax = fig.add_axes([0.2,0.10,0.75,0.8])
# ax_inset = fig.add_axes([0.35,0.36,0.4,0.5])

ax.plot(np.linspace(-0.1,0.1,41),Error_one_5ns*1,label='n=1',linewidth=3)
ax.plot(np.linspace(-0.1,0.1,21),Error_30_5ns*1,label='n=1',linewidth=3)
# ax.plot(np.linspace(-0.1,0.1,21),Error_09_5ns*1,label='n=1',linewidth=3)
ax.plot(np.linspace(-0.1,0.1,21),Error_60ns_09_5ns*1,label='n=1',linewidth=3)

# ax.plot(np.linspace(-0.1,0.1,41),Error_two_5ns*1,label='n=2',linewidth=3)

ax.set_xlim([-0.105,0.105])
# ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))

ax.tick_params(axis='x',which='both',bottom=True,top=True,direction='in' )
ax.tick_params(axis='y',which='both',left=True,right=True,direction='in' )

ax.tick_params(labelsize=16)
plt.tick_params(labelsize=16)
ax.set_yscale('log')

In [48]:
%matplotlib qt

In [55]:
gate_time=60e-9
SRATE=10e9
f_term=2
factor_r=-0.06
T_reflex=5e-9

factor1=1.0 
factorc=1.0 
factor2=1.0 

lamb1 = [0.8,-0.1]
total_len = gate_time + 8e-9+4*T_reflex

Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
Seq.clear_pulse(tips_on=False)
Seq.add_pulse('Adiabatic',t0=gate_time/2+10e-9/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=f_term,Lcoeff=np.array(lamb1),Q1_freq=6.0e9,
             CPLR_idle_freq=(7.87e9-6e9)*factorc+6e9,Q2_freq=6e9+(5.4e9-6e9)*factor2,constant_coupling=False,r1c=0.018*factor1,r2c=0.018*factor2,r12=0.0015*factorc,anhar_CPLR=-300e6*factorc,
             anhar_Q1=-250e6*factor1,anhar_Q2=-250e6*factor2,negative_amplitude=False,dfdV=(7.87e9-6e9)*factorc+6e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
Seq.add_filter('Gauss Low Pass',300e6)
Seq.add_filter('Reflection',*(factor_r,T_reflex))
flux_pulse=Seq.get_sequence()

plt.plot(flux_pulse)

## compare to coupler-free structrue

In [ ]:
raw_config = Circuit_Simulator.RawConfig(qubit_num=2,dimension=3,circuit_type=1,initial_state='ground',sampling_rate=1e9)
raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)

flux_pulse = np.linspace(0/7.5,4/7.5,400)
freq_array = 8.5e9-flux_pulse * (8.5e9 - 1e9)

raw_config.setValue('Q1 f01_max',6.0e9)
raw_config.setValue('Q2 f01_max',8.0e9)
raw_config.setValue('Q1 f01_min',1e9)
raw_config.setValue('Q2 f01_min',1e9)
raw_config.setValue('Q1 Ec',0.25e9)
raw_config.setValue('Q2 Ec',0.30e9)

raw_config.setValue('r12',0.008)

raw_config.setValue('Q2 Voltage period',-1)
raw_config.setValue('Q2 Voltage operating point',0.00)
raw_config.setValue('Q2 Flux',flux_pulse)

simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
Simulator = Circuit_Simulator.Simulator(simu_config)
Simulator.show_pulse()

In [ ]:
Simulator.performsimulation(solver_type=2,resample_factor=1,eigen_cloest_to_bare=False ,sort_by_maximum_overlap=True,gap=10e6)

fig = plt.figure(figsize=[6.4,6])
ax = fig.add_subplot(111)

eigen_trace = Simulator.EigenResult.get_Ener_gap_trace('11-10-01+00')
ax.plot(freq_array[0:400],-eigen_trace[0:400]/1e6)

ax.set_yscale('log')

In [ ]:
%matplotlib inline

In [ ]:
gate_time,SRATE,f_term,factor_r,T_reflec=(40e-9,10e9,1,0,2e-9)
lamb1 = 0.8
total_len = gate_time + 8e-9+4*T_reflec


Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
Seq.clear_pulse(tips_on=False)
Seq.add_pulse('Slepian',t0=gate_time/2+10e-9/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=f_term,Lcoeff=np.array(lamb1),
              Coupling=20e6,Offset=1e9,dfdV=5e9,negative_amplitude=None)
Seq.add_filter('Gauss Low Pass',300e6)
Seq.add_filter('Reflection',*(factor_r,T_reflec))
flux_pulse=Seq.get_sequence()

plt.plot(flux_pulse)

In [ ]:

def cost_func_distor_18(pulse_params,*args):
    gate_time,SRATE,f_term,factor_r,T_reflec=args
    lamb1 = pulse_params
    total_len = gate_time + 8e-9+4*T_reflec
    
    Seq=PG.Sequence(total_len=total_len,sample_rate=SRATE,complex_trace=False)
    Seq.clear_pulse(tips_on=False)
    Seq.add_pulse('Adiabatic',t0=gate_time/2+10e-9/2,width=gate_time,plateau=0e-9,frequency=0,F_Terms=f_term,Lcoeff=np.array(lamb1),Q1_freq=6.0e9,
                 CPLR_idle_freq=7.87e9,Q2_freq=5.4e9,constant_coupling=False,r1c=0.018,r2c=0.018,r12=0.0015,anhar_CPLR=-300e6,
                 anhar_Q1=-250e6,anhar_Q2=-250e6,negative_amplitude=False,dfdV=7.87e9-1e9,gap_threshold=8e6,freqpoints=301,pulsepoints=601)
    Seq.add_filter('Gauss Low Pass',300e6)
    Seq.add_filter('Reflection',*(factor_r,T_reflec))
    flux_pulse=Seq.get_sequence()

    raw_config = Circuit_Simulator.RawConfig(qubit_num=3,dimension=3,circuit_type=1,initial_state='-Z+Z+Z',sampling_rate=SRATE)
    raw_config.load_default_value(modulation=True,decoherence=False,use_capacitance=False)
    
    raw_config.setValue('Q1 f01_max',6.0e9)
    raw_config.setValue('Q2 f01_max',7.87e9)
    raw_config.setValue('Q3 f01_max',5.4e9)
    raw_config.setValue('Q1 f01_min',1e9)
    raw_config.setValue('Q2 f01_min',1e9)
    raw_config.setValue('Q3 f01_min',1e9)
    raw_config.setValue('Q1 Ec',0.25e9)
    raw_config.setValue('Q2 Ec',0.3e9)
    raw_config.setValue('Q3 Ec',0.25e9)
    raw_config.setValue('r12',0.018)
    raw_config.setValue('r23',0.018)
    raw_config.setValue('r13',0.0015)
    raw_config.setValue('Q2 Voltage period',-1)
    raw_config.setValue('Q2 Voltage operating point',0)
    raw_config.setValue('Q2 Flux',flux_pulse)

    simu_config = Circuit_Simulator.read_config(raw_config.get_dict())
    Simulator = Circuit_Simulator.Simulator(simu_config)

    Simulator.performsimulation(solver_type=1)
    Simulator.UnitaryResult.get_U(-1)
    Simulator.UnitaryResult.get_subspace_operator(['000','001','100','101'])
    Simulator.UnitaryResult.set_Target_gate('CZ')
    Simulator.UnitaryResult.remove_single_qubit_gate()
    Simulator.UnitaryResult.get_Gate_Fidelity()
    fidelity = Simulator.UnitaryResult.Gate_Fidelity
    return 1 - fidelity